In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from helper.constantes import *
import numpy as np
import pandas as pd

We start by reading the file containing all the spells from the HP Collection database. 

In [4]:
df_spell = pd.read_json(hpcollection_folder+"spells.json")
df_spell.head(5)

,id,name,other_name,pronunciation,spell_type,description,mention,etymology,note
0,1,Aberto,None,Ah-bare-toh,Charm,None,None,None,None
1,2,Accio,Summoning Charm,"Various, including: AK-ee-oh or AK-see-oh , AK...",Charm,Summons an object towards the caster. It is ab...,Harry Potter summoned his broom to complete th...,"The Latin word accio means ""I call"" or ""I summ...",None
2,3,Age Line,None,None,Charm,Prevents people above or below a certain age f...,None,None,None
3,4,Aguamenti,Water-Making Spell,AH-gwah-MEN-tee,"Charm, Conjuration","Produces a clean, drinkable jet of water from ...",Used by Fleur Delacour in 1994 to extinguish h...,None,None
4,5,Alarte Ascendare,None,a-LAR-tay a-SEN-der-ay,Charm,Shoots the target high into the air.,Used by Gilderoy Lockhart in 1992 to send a sn...,None,None


To count the number of occurence, we start by declaring a dictionnary that will contains all the spell. We take advantage of this opportunity to fill in a set with all the spells and also fill in a dictionnary mapping the spell name to the spell type. 

In [5]:
# Accumulator
occ = {}
spells = set()
spell_type = {}
# Iterate over all the spells in the dataframe
for i, v in df_spell.iterrows():
    name = v['name'].lower()
    occ[name]= 0
    spells.add(name)
    spell_type[name] = v['spell_type']

# Remove the spell pack as it is almost never used in the saga and a lot of word contains 'pack' in their spelling.
spells.remove("pack")

In [6]:
# Create a list with all the lines in the Harry Potter books
all_lines = []
cur_line = 1
for i in range(1,8):
    # Read the full book line by line in the a list
    with open(data_folder+hpbooks_folder+f"hp{i}.txt") as f:
        lines = f.readlines()
    cur_str = ""
    for line in lines:
        # Line in the text file!= line in the book, i.e. some line in the book might spend several line in the text file  
        if line == "\n":
            # if the line is \n, we append the line to the list and reset the counter
            all_lines.append((cur_line, i, cur_str.replace("  ", " ").lower()))
            cur_str = ""
            cur_line += 1
        else:
            # Otherwise we concatenate the sprint to the already existing one
            cur_str = (cur_str + " " if cur_str != "" else "") + line.replace("\”", "'").replace("“","'").replace("”","'").replace("\n","").strip()

In [7]:
occurences = []
# Build the list of casted spell
for lineCount, book_id, line in all_lines:
    # Check if any of the spell occured in the sentence. 
    for sp in spells: 
        if sp in line:
            # Append the spell to the list of occurences.
            occurences.append((lineCount, book_id, sp,spell_type[sp], line))
            occ[sp] = occ[sp] +1

In [8]:
import csv

# Save The files on disk line by line.
with open(cleaned_folder+ 'spellByBooke.csv','w') as out:
    csv_out=csv.writer(out)
    csv_out.writerow(['lineNb','bookId','spell','spellType', 'sentence'])
    for row in occurences:
        csv_out.writerow(row)